<a href="https://colab.research.google.com/github/ArnaudOlt/dissertation_pybamm/blob/main/dev_read_lfp_balance_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%pip install pybtex
%pip install anytree
%pip install autograd
%pip install bpx
%pip install casadi
%pip install imageio
%pip install importlib-metadata
%pip install matplotlib
%pip install numpy
%pip install pandas
%pip install scikit-fem
%pip install scipy
%pip install sympy
%pip install tqdm
%pip install xarray
%pip install git+https://github.com/pybamm-team/pybamm.git@develop

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 561.4/561.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.1/160.1 kB 3.3 MB/s eta 0:00:00
  Cloning https://github.com/pybamm-team/pybamm.git (to revision develop) to /tmp/pip-req-build-c0dvq3j_
  Running command git clone --filter=blob:none --quiet https://github.com/pybamm-team/pybamm.git /tmp/pip-req-build-c0dvq3j_
  Resolved https://github.com/pybamm-team/pybamm.git to commit b6d4f3f09fae887d142e52913219595b11098c3d
  Preparing metadata (setup.py) ... done


In [ ]:
import pybamm
import numpy as np

In [ ]:
model = pybamm.lithium_ion.DFN(
    {
        #"SEI": "solvent-diffusion limited",
        #"SEI porosity change": "true",
        "lithium plating": "partially reversible",
        "lithium plating porosity change": "true",  # alias for "SEI porosity change"
        #"particle mechanics": ("swelling and cracking", "swelling only"),
        #"SEI on cracks": "true",
        #"loss of active material": "stress-driven",
        #"calculate discharge energy": "true",  # for compatibility with older PyBaMM versions
    }
)

In [ ]:
parameter_values = pybamm.ParameterValues("OKane2022")

In [ ]:
def LFP_ocp_Afshar2017(sto):
    """
    Open-circuit potential for LFP. Prada2013 doesn't give an OCP for LFP, so we use
    Afshar2017 instead.

    References
    ----------
    .. [1] Afshar, S., Morris, K., & Khajepour, A. (2017). Efficient electrochemical
    model for lithium-ion cells. arXiv preprint arXiv:1709.03970.

    Parameters
    ----------
    sto : :class:`pybamm.Symbol`
       Stochiometry of material (li-fraction)

    """

    c1 = -150 * sto
    c2 = -30 * (1 - sto)
    k = 3.4077 - 0.020269 * sto + 0.5 * np.exp(c1) - 0.9 * np.exp(c2)

    return k

def LFP_electrolyte_exchange_current_density_kashkooli2017(c_e, c_s_surf, c_s_max, T):
    """
    Exchange-current density for Butler-Volmer reactions between LFP and electrolyte

    References
    ----------
    .. [1] Kashkooli, A. G., Amirfazli, A., Farhad, S., Lee, D. U., Felicelli, S., Park,
    H. W., ... & Chen, Z. (2017). Representative volume element model of lithium-ion
    battery electrodes based on X-ray nano-tomography. Journal of Applied
    Electrochemistry, 47(3), 281-293.

    Parameters
    ----------
    c_e : :class:`pybamm.Symbol`
        Electrolyte concentration [mol.m-3]
    c_s_surf : :class:`pybamm.Symbol`
        Particle concentration [mol.m-3]
    c_s_max : :class:`pybamm.Symbol`
        Maximum particle concentration [mol.m-3]
    T : :class:`pybamm.Symbol`
        Temperature [K]

    Returns
    -------
    :class:`pybamm.Symbol`
        Exchange-current density [A.m-2]
    """

    m_ref = 6 * 10 ** (-7)  # (A/m2)(m3/mol)**1.5 - includes ref concentrations
    E_r = 39570
    arrhenius = np.exp(E_r / pybamm.constants.R * (1 / 298.15 - 1 / T))

    return (
        m_ref * arrhenius * c_e**0.5 * c_s_surf**0.5 * (c_s_max - c_s_surf) ** 0.5
    )

# positive electrode
parameter_values.update({"Positive electrode conductivity [S.m-1]": 0.33795074}) # prada2013
parameter_values.update({"Maximum concentration in positive electrode [mol.m-3]": 22806.0}) # prada2013
parameter_values.update({"Positive electrode diffusivity [m2.s-1]":  5.9e-18}) # prada2013
parameter_values.update({"Positive electrode OCP [V]": LFP_ocp_Afshar2017}, check_already_exists=False) # prada2013
parameter_values.update({"Positive electrode porosity": 0.426}) # prada2023
parameter_values.update({"Positive electrode active material volume fraction": 0.374}) # prada2013
parameter_values.update({"Positive particle radius [m]": 5e-08}) # prada2013
parameter_values.update({"Positive electrode Bruggeman coefficient (electrode)": 1.5})
parameter_values.update({"Positive electrode Bruggeman coefficient (electrolyte)": 1.5})
parameter_values.update({"Positive electrode charge transfer coefficient": 0.5})
parameter_values.update({"Positive electrode double-layer capacity [F.m-2]": 0.2})
parameter_values.update({"Positive electrode exchange-current density [A.m-2]": LFP_electrolyte_exchange_current_density_kashkooli2017}, check_already_exists=False) # prada 2013
parameter_values.update({"Positive electrode OCP entropic change [V.K-1]": 0}) # prada2013


In [ ]:
pybamm.set_logging_level("NOTICE")
cycle_number = 50
exp = pybamm.Experiment(
    ["Hold at 4.2 V until C/100", #change at C/20 effects speed
    "Rest for 4 hours",
    "Discharge at 0.1C until 2.5 V",  # initial capacity check
    "Charge at 0.3C until 4.2 V",
    "Hold at 4.2 V until C/100",]
    + [("Discharge at 1C until 2.5 V",  # ageing cycles
    "Charge at 0.3C until 4.2 V",
    "Hold at 4.2 V until C/100",)] * cycle_number
    + ["Discharge at 0.1C until 2.5 V"],  # final capacity check
    period="5 minutes"
)


In [ ]:
sim = pybamm.Simulation(model, parameter_values=parameter_values, experiment=exp)

# safe solver and reduced dt_max
safe_solver = pybamm.CasadiSolver(mode="safe", dt_max=6)

# solve
sol = sim.solve(solver=safe_solver, calc_esoh=False)


2023-08-09 10:40:36.265 - [NOTICE] callbacks.on_cycle_start(172): Cycle 1/56 (124.093 us elapsed) --------------------
NOTICE:pybamm.logger:Cycle 1/56 (124.093 us elapsed) --------------------
2023-08-09 10:40:36.269 - [NOTICE] callbacks.on_step_start(180): Cycle 1/56, step 1/1: Hold at 4.2 V until C/100
NOTICE:pybamm.logger:Cycle 1/56, step 1/1: Hold at 4.2 V until C/100
2023-08-09 10:40:38.286 - [NOTICE] callbacks.on_cycle_start(172): Cycle 2/56 (2.022 s elapsed) --------------------
NOTICE:pybamm.logger:Cycle 2/56 (2.022 s elapsed) --------------------
2023-08-09 10:40:38.291 - [NOTICE] callbacks.on_step_start(180): Cycle 2/56, step 1/1: Rest for 4 hours
NOTICE:pybamm.logger:Cycle 2/56, step 1/1: Rest for 4 hours
The linesearch algorithm failed with too small a step.
2023-08-09 10:40:40.161 - [NOTICE] callbacks.on_cycle_start(172): Cycle 3/56 (3.896 s elapsed) --------------------
NOTICE:pybamm.logger:Cycle 3/56 (3.896 s elapsed) --------------------
2023-08-09 10:40:40.163 - [NOTIC

In [12]:
git commit -m "first commit"

SyntaxError: ignored